# ELUC Predictor
Train an XGBoost model that can predict ELUC

In [ ]:
# %pip install zarr
# %pip install regionmask
# %pip install xgboost
# %pip install scikit-learn

In [ ]:
import numpy as np
import pandas as pd
import regionmask
import xarray as xr
import matplotlib.pyplot as plt

In [ ]:
ds = xr.open_zarr("../data/gcb/processed/merged_aggregated_dataset_1850_2022.zarr.zip", consolidated=True)

# Data prep
## Shift ELUC by 1 year
According to the [ELUC Data Exploration](../data/gcb/ELUC_data_exploration.ipynb) notebook ELUC is off by one year when compared to the change in land use. Shifting it by 1 year here.

In [ ]:
ds['ELUC'] = ds['ELUC'].roll(time=1, roll_coords=False)
ds['ELUC_diff'] = ds['ELUC_diff'].roll(time=1, roll_coords=False)

## Add countries

In [ ]:
ds = ds.stack(latlon=('lat', 'lon'))

In [ ]:
regionmask.__version__

In [ ]:
# Train on countries 0-80, and test on the rest, and test on the last 15 years of data
country_mask = regionmask.defined_regions.natural_earth_v5_0_0.countries_110.mask(ds)
ds = ds.assign_coords(country=country_mask)

In [ ]:
ds

In [ ]:
countries = regionmask.defined_regions.natural_earth_v5_0_0.countries_110
countries

In [ ]:
ds.country

In [ ]:
np.unique(ds.time)

## Remove NaN ELUC_diff

In [ ]:
# Create a boolean mask based on NaN values for ELUC_diff
mask = ds['ELUC_diff'].isnull()

In [ ]:
# Filter the dataset based on the mask
ds = ds.where(~mask, drop=True)

## Train and test sets
Create a dataset for a single country to keep it small and easy to train on  
Country: United Kingdom  
Train on data between 1850 and 2007  
Test on data after 2007 (i.e. 2008 to 2021)

In [ ]:
countries_df = countries.to_dataframe()
countries_df

In [ ]:
countries_df[countries_df.names == 'United Kingdom']

In [ ]:
# train_da = dataset.where(dataset.country <= 80, drop=True).where(dataset.time <= 2007, drop=True)
# test_da = dataset.where(dataset.country > 80,  drop=True).where(dataset.time > 2007, drop=True)
train_da = ds.where(ds.country == 143, drop=True).where(ds.time <= 2007, drop=True)
test_da = ds.where(ds.country == 143,  drop=True).where(ds.time > 2007, drop=True)

In [ ]:
train_da

In [ ]:
test_da

## Features

In [ ]:
features = list(train_da.data_vars)
# This is the target
features.remove('ELUC_diff')
features.remove('ELUC')
# Do not use:
features.remove('cell_area')
features.remove('cell_area_diff')

In [ ]:
features

In [ ]:
# Extract features and target variable from the training dataset
X_train = train_da[features]
# y_train = train_da['ELUC_diff']
y_train = train_da['ELUC']

In [ ]:
# Extract features and target variable from the test dataset
X_test = test_da[features]
# y_test = test_da['ELUC_diff']
y_test = test_da['ELUC']

In [ ]:
X_test

In [ ]:
y_test

In [ ]:
def convert_to_dataframe(da):
    df = da.to_dataframe()
    df.index = df.index.set_names(['time', 'i_lat', 'i_lon'])
    df = df.reset_index()
    return df

In [ ]:
X_test_df = convert_to_dataframe(X_test)

In [ ]:
X_test_df.tail(10)

In [ ]:
y_test_df = convert_to_dataframe(y_test)
y_test_df.tail()

In [ ]:
np.isnan(y_test.values)

# XGBoost (xarray)

In [ ]:
# import xgboost as xgb
# from sklearn.metrics import mean_squared_error
# from sklearn.metrics import mean_absolute_error

In [ ]:
# X_train = np.column_stack([X_train[feature].values for feature in features])

In [ ]:
# model = xgb.XGBRegressor()

In [ ]:
# %%time
# model.fit(X_train, y_train)

In [ ]:
# X_test = np.column_stack([X_test[feature].values for feature in features])

In [ ]:
# preds = model.predict(X_test)
# preds

In [ ]:

# mse = mean_squared_error(y_test, preds, squared=True)
# rmse = mean_squared_error(y_test, preds, squared=False)
# mae = mean_absolute_error(y_test, preds)
# print(f"MSE: {mse}")
# print(f"RMSE: {rmse}")
# print(f"MAE: {mae}")

In [ ]:
# preds.shape

# XGBoost Train (df)

In [ ]:
import xgboost as xgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

In [ ]:
predictor = xgb.XGBRegressor()

In [ ]:
X_train_df = convert_to_dataframe(X_train)
y_train_df = convert_to_dataframe(y_train)

In [ ]:
%%time
predictor.fit(X_train_df[features], y_train_df["ELUC"])

In [ ]:
preds = predictor.predict(X_test_df[features])

In [ ]:
preds.shape

In [ ]:
# y_test_df = y_test_df["ELUC"]

In [ ]:
mse = mean_squared_error(y_test_df["ELUC"], preds, squared=True)
rmse = mean_squared_error(y_test_df["ELUC"], preds, squared=False)
mae = mean_absolute_error(y_test_df["ELUC"], preds)
print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"MAE: {mae}")

In [ ]:
feature_important = predictor.get_booster().get_score(importance_type='weight')
keys = list(feature_important.keys())
values = list(feature_important.values())

feature_importance_df = pd.DataFrame(data=values, index=keys, columns=["score"]).sort_values(by = "score", ascending=False)
feature_importance_df.nlargest(40, columns="score").plot(kind='barh', figsize = (20,10)) ## plot top 40 features

# Evaluate
## Aggregate at the country level

In [ ]:
LAND_FEATURES = ['c3ann', 'c3nfx', 'c3per','c4ann', 'c4per',
 'pastr', 'primf', 'primn', 'range', 'secdf', 'secdn', 'urban']
LAND_DIFF_FEATURES = ['c3ann_diff', 'c3nfx_diff', 'c3per_diff','c4ann_diff', 'c4per_diff',
 'pastr_diff', 'primf_diff', 'primn_diff', 'range_diff', 'secdf_diff', 'secdn_diff', 'urban_diff']

In [ ]:
agg_df = X_test_df \
        .groupby(["time"])[LAND_DIFF_FEATURES] \
        .mean() \
        .sort_values(by=["time"]).reset_index()

In [ ]:
agg_df[["time"] + LAND_DIFF_FEATURES].plot(title="Mean LUC in UK", x='time')
plt.legend(bbox_to_anchor=(1.0, 1.0))

In [ ]:
uk_y_df = y_test_df

In [ ]:
uk_preds = predictor.predict(X_test_df[features])

In [ ]:
uk_y_df["ELUC_Pred"] = uk_preds

In [ ]:
agg_y_df = uk_y_df \
        .groupby(["time"])[["ELUC", "ELUC_Pred"]] \
        .mean() \
        .sort_values(by=["time"]).reset_index()

In [ ]:
agg_y_df.plot(x="time", y=["ELUC", "ELUC_Pred"], title="ELUC vs Predicted ELUC in UK")

## Single point

In [ ]:
# Max ELUC
y_test_df["ELUC"].max()

In [ ]:
y_test_df[y_test_df["ELUC"] == y_test_df["ELUC"].max()]

In [ ]:
wales_y_df = y_test_df[(y_test_df.i_lat==52.875) & 
                       (y_test_df.i_lon==-4.625)]
wales_y_df

In [ ]:
wales_X_df = X_test_df[(X_test_df.i_lat==52.875) & 
                       (X_test_df.i_lon==-4.625)]
wales_X_df

In [ ]:
wales_X_df[["time"] + LAND_DIFF_FEATURES].plot(x='time')
plt.legend(bbox_to_anchor=(1.0, 1.0))

In [ ]:
wales_y_df[['ELUC', 'time']].plot(x="time", y="ELUC")

In [ ]:
wales_preds = predictor.predict(wales_X_df)

In [ ]:
wales_preds.shape

In [ ]:
wales_y_df["ELUC_Pred"] = wales_preds

In [ ]:
wales_y_df

In [ ]:
wales_y_df.plot(x="time", y=["ELUC", "ELUC_Pred"])